# Balanced Prediction

In [68]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
url = "https://raw.githubusercontent.com/juliasweet/Final_Project/SweetJ/Success_Only2.csv"
success_only = pd.read_csv(url)
success_only.head()

,Unnamed: 0,Unnamed: 0.1,shotgun,no_huddle,ydstogo,yardline_100,play_type,half_seconds_remaining,down,score_differential,fg_prob,td_prob
0,0,0,1,0,10,39.0,pass,1681.0,3.0,0.0,0.415009,0.333712
1,3,3,1,0,5,39.0,run,1404.0,3.0,0.0,0.395592,0.377457
2,7,7,1,0,7,57.0,pass,773.0,3.0,-3.0,0.269825,0.319773
3,8,8,1,1,1,22.0,run,613.0,3.0,-3.0,0.415125,0.454985
4,14,14,0,0,1,54.0,run,1429.0,3.0,-3.0,0.260790,0.458135


In [69]:
success_only['play_type'].value_counts()

pass    13473
run      5029
Name: play_type, dtype: int64

In [31]:
13470 / (13470 + 5000.)

0.7292907417433676

In [71]:
success_only[success_only['ydstogo'] <= 3]['play_type'].value_counts()

run     3824
pass    3627
Name: play_type, dtype: int64

In [42]:
url = "https://raw.githubusercontent.com/juliasweet/Final_Project/SweetJ/thirdDown.csv"
Third_Down = pd.read_csv(url)
Third_Down.head()

,Unnamed: 0,play_id,game_id,shotgun,no_huddle,ydstogo,yardline_100,play_type,half_seconds_remaining,down,pass_length,pass_length.1,run_location,score_differential,fg_prob,td_prob,ep,third_down_converted
0,0,190,2018090600,1,0,10,39.0,pass,1681.0,3.0,deep,deep,NaN,0.0,0.415009,0.333712,2.250232,1.0
1,1,256,2018090600,1,0,1,1.0,pass,1559.0,3.0,short,short,NaN,0.0,0.204515,0.749825,5.661931,0.0
2,2,344,2018090600,1,0,4,92.0,pass,1464.0,3.0,short,short,NaN,0.0,0.093928,0.213978,-1.832560,0.0
3,3,445,2018090600,1,0,5,39.0,run,1404.0,3.0,NaN,NaN,right,0.0,0.395592,0.377457,2.642543,1.0
4,4,636,2018090600,1,0,3,3.0,pass,1207.0,3.0,short,short,NaN,0.0,0.386222,0.567801,4.926552,0.0


In [55]:
Third_Down['play_type'].value_counts()

pass    37025
run      9852
Name: play_type, dtype: int64

In [61]:
import seaborn as sns
sns.countplot(x='play_type', Third_Down=Third_Down, palette='hls')
ple.show()
ple.savefig('count_plot')

ValueError: Could not interpret input 'play_type'

In [44]:
run_plays = Third_Down[Third_Down['play_type'] == 'run']
run_plays.shape

(9852, 18)

In [45]:
run_plays[run_plays['ydstogo'] > 40]

,Unnamed: 0,play_id,game_id,shotgun,no_huddle,ydstogo,yardline_100,play_type,half_seconds_remaining,down,pass_length,pass_length.1,run_location,score_differential,fg_prob,td_prob,ep,third_down_converted
21129,21129,3785,2014110905,0,0,46,78.0,run,105.0,3.0,NaN,NaN,right,14.0,0.101425,0.063212,0.034093,0.0


In [35]:
run_plays['ydstogo'].describe()

count    9852.000000
mean        4.933821
std         5.442391
min         1.000000
25%         1.000000
50%         2.000000
75%         7.000000
max        46.000000
Name: ydstogo, dtype: float64

In [46]:
y.value_counts()

pass    37025
run      9852
Name: play_type, dtype: int64

In [47]:
37025. / (37025 + 9852.)

0.7898329671267359

In [48]:
Third_Down[Third_Down['ydstogo'] <= 7].shape

(27628, 18)

In [51]:
#Define X & Y From Right side
X = Third_Down[['ydstogo','half_seconds_remaining', 'score_differential', 'yardline_100','fg_prob', 'td_prob']]
y = Third_Down['play_type']

# Split into test and train
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, stratify=y)
Third_Down.dtypes

Unnamed: 0                  int64
play_id                     int64
game_id                     int64
shotgun                     int64
no_huddle                   int64
ydstogo                     int64
yardline_100              float64
play_type                  object
half_seconds_remaining    float64
down                      float64
pass_length                object
pass_length.1              object
run_location               object
score_differential        float64
fg_prob                   float64
td_prob                   float64
ep                        float64
third_down_converted      float64
dtype: object

In [52]:
from sklearn.linear_model import LogisticRegression
logmodel = LogisticRegression()
logmodel.fit(X_train,y_train)
predictions = logmodel.predict(X_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [53]:
from sklearn.metrics import classification_report
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

        pass       0.79      1.00      0.88      9257
         run       0.71      0.03      0.06      2463

   micro avg       0.79      0.79      0.79     11720
   macro avg       0.75      0.51      0.47     11720
weighted avg       0.78      0.79      0.71     11720



#The f1-score gives you the harmonic mean of precision and recall
#The scores corresponding to every class will tell you the accuracy of the classifier in classifying the data points in that particular class compared to all other classes
#The support is the number of samples of the true response that lie in that class

In [54]:
# Create the logistic regression
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier
classifier.fit(X_train, y_train)
print(f"First 10 predictions: {predictions[:10]}")
print(f"First 10 Actual Labels: {y_test[:10].tolist()}")
pd.DataFrame({"Prediction:": predictions, "Actual:": y_test}).reset_index(drop=True)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


First 10 predictions: ['pass' 'pass' 'pass' 'pass' 'pass' 'pass' 'pass' 'pass' 'pass' 'pass']
First 10 Actual Labels: ['pass', 'pass', 'pass', 'pass', 'pass', 'pass', 'pass', 'pass', 'pass', 'pass']


,Prediction:,Actual:
0,pass,pass
1,pass,pass
2,pass,pass
3,pass,pass
4,pass,pass
5,pass,pass
6,pass,pass
7,pass,pass
8,pass,pass
9,pass,pass


In [12]:
from sklearn.metrics import confusion_matrix

In [13]:
confusion_matrix(y_test, predictions)

array([[9224,   33],
       [2383,   80]], dtype=int64)

In [18]:
from sklearn.metrics import accuracy_score

In [20]:
accuracy_score(y_test, predictions)

0.7938566552901024

# Weighted Prediction


In [64]:

#Define X & Y From Right side
X = Third_Down[['ydstogo','half_seconds_remaining', 'score_differential', 'yardline_100','fg_prob', 'td_prob']]
y = Third_Down['play_type']

# Split into test and train
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, stratify=y)
Third_Down.dtypes

Unnamed: 0                  int64
play_id                     int64
game_id                     int64
shotgun                     int64
no_huddle                   int64
ydstogo                     int64
yardline_100              float64
play_type                  object
half_seconds_remaining    float64
down                      float64
pass_length                object
pass_length.1              object
run_location               object
score_differential        float64
fg_prob                   float64
td_prob                   float64
ep                        float64
third_down_converted      float64
dtype: object

In [66]:
from sklearn.linear_model import LogisticRegression
weights = {"td_prob":0.155520, "ydstogo": 0.147366,"half_seconds_remaining":0.130070,
"shotgun": 0.129944, "ep":  0.125123, "fg_prob": 0.122270, "score_differential": 0.096070,
"yardline_100":0.073235,"third_down_converted": 0.013885,"no_huddle": 0.006516}
logmodel = LogisticRegression(class_weight=weights)
logmodel.fit(X_train,y_train)
predictions = logmodel.predict(X_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


ValueError: Class label td_prob not present.

In [ ]:
# Create the logistic regression
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier
classifier.fit(X_train, y_train)
print(f"First 10 predictions: {predictions[:10]}")
print(f"First 10 Actual Labels: {y_test[:10].tolist()}")
pd.DataFrame({"Prediction:": predictions, "Actual:": y_test}).reset_index(drop=True)